In [1]:
import os
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep
from tqdm import tqdm_notebook as tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh


# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [14]:
model = Model("./data/1kabc/simple/train/00070090_73b2f35a88394199b6fd1ab8_003.obj")
model.generate_view_points(10)

model.illustrate()

Output()

CPU times: user 217 ms, sys: 4.02 ms, total: 221 ms
Wall time: 53.1 ms
CPU times: user 212 ms, sys: 3.88 ms, total: 216 ms
Wall time: 51.7 ms
CPU times: user 201 ms, sys: 11.8 ms, total: 213 ms
Wall time: 51.2 ms
CPU times: user 214 ms, sys: 8.14 ms, total: 222 ms
Wall time: 53.9 ms
CPU times: user 232 ms, sys: 11.9 ms, total: 243 ms
Wall time: 58.2 ms
CPU times: user 220 ms, sys: 8 µs, total: 220 ms
Wall time: 52.9 ms
CPU times: user 208 ms, sys: 7.87 ms, total: 215 ms
Wall time: 51.9 ms
CPU times: user 233 ms, sys: 0 ns, total: 233 ms
Wall time: 56 ms
CPU times: user 224 ms, sys: 0 ns, total: 224 ms
Wall time: 54.5 ms
CPU times: user 241 ms, sys: 4.13 ms, total: 245 ms
Wall time: 59 ms
CPU times: user 43.8 s, sys: 1.21 s, total: 45 s
Wall time: 7 s
0.46520023731505894


In [2]:
NUM_POINTS = 10


plot = k3d.plot(name='points')
plot.display()

model = Model("./data/Pyramid.obj", resolution_image=64)
model.generate_view_points(NUM_POINTS)

observations = []
for view_point_idx in range(NUM_POINTS):
    %time observation = model.get_observation(view_point_idx)
    
    plot = observation.illustrate(plot, size=0.04)
    plot = illustrate_points([model.get_point(view_point_idx)], size=1.0, plot=plot)
#     sleep(2)
    
    observations.append(observation)
    
combined_observation = combine_observations(observations)
%time reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
print(loss)

# illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Output()

KeyboardInterrupt: 

In [8]:
illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [16]:
len(combined_observation.points), loss

(755551, 0.08265528953648774)

In [9]:
len(combined_observation.points), loss

(47203, 0.2490280261287171)

In [10]:
%%time
reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)
loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)

CPU times: user 19.3 s, sys: 129 ms, total: 19.4 s
Wall time: 3.08 s


In [ ]:
NUM_POINTS = 10

models_path = "./data/1kabc/simple/val"

losses = []
for model_name in tqdm(sorted(os.listdir(models_path))):
    model = Model(os.path.join(models_path, model_name))
    model.generate_view_points(NUM_POINTS)

    combined_observation = None
    for view_point_idx in range(NUM_POINTS):
        observation = model.get_observation(view_point_idx)

        if combined_observation is None:
            combined_observation = observation
        else:
            combined_observation += observation
    
    reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

    loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
    print(loss)
    losses.append(loss)

0.43260879503679267
0.011748805825769773
0.6368368930715143
0.01999495878259948
0.009864535577272508
0.0049465034012368585
0.004867339413804205
2.114583249307729
5.513358045365431
0.007657026293620387
0.012763651915830367
0.022809397706465888
0.606697827210757
0.021164343424414495
0.008369065356445203
0.6542076437755041
0.008919581879375504
0.008961263241012722
0.9702387490236133
1.3275972704264303
